In [1]:
def WSI_CP(root, des, CPI):
    import glob,os,imagecodecs,tifffile as tiff,shutil
    from patchify import patchify 

    des = des+'WSI_CP'
    if os.path.isdir(des) is False:
        os.makedirs(des)
    else:
        shutil.rmtree(des)
        os.makedirs(des)

    CPI_filenames = os.listdir(CPI)

    for file in glob.glob(root+'*.tif'):
        file = file.replace("\\","/")
        img = tiff.imread(file)

        filename = file[len(root):-4]
        to_read_file = ''
        for i in range(len(CPI_filenames)):
            if filename == CPI_filenames[i].split('_')[0]:
                to_read_file = CPI_filenames[i]
                CPI_filenames.pop(i)
                break

        

        patches = patchify(img, (256, 256,3), step=256)
        x, y, z = patches.shape[0], patches.shape[1], patches.shape[2]
        patch = []

        for i in range(x):
            for j in range(y):
                for k in range(z):
                    patch.append(patches[i][j][k])
        
        CP = []
        with open(CPI+to_read_file) as file:
            for i in file.readlines():
                CP.append(int(i))


        for i in CP:
            tiff.imwrite(des+'/{}_CP_{}.jpg'.format(filename,i), patch[i], photometric='rgb')
        
        if not CPI_filenames:
            break
    
    # !zip -r '/content/patches' '/content/patches'
    # files.download('{}.zip'.format(filename))

In [5]:
root = 'H:/Azib/Lung_cancer/LCDC/Train/'
des = 'H:/Azib/Lung_cancer/LCDC/'
CPI = 'H:/Azib/Lung_cancer/LCDC/cancer_patch_id/'

In [6]:
WSI_CP(root,des,CPI)